In [ ]:
import pandas as pd
dfg = pd.read_excel("galena1 (4) (1).xlsx")
dfo= pd.read_excel("olivine  (1).xlsx")
print(dfg.shape,dfo.shape)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt

# Load and preprocess your data


# Drop unnecessary columns and handle missing values
selected_columns = ['U_corrected_ppm', 'Th_corrected_ppm']  # Example columns
data = dfg[selected_columns].dropna()

# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# Define the Autoencoder model
input_layer = Input(shape=(len(selected_columns),))
encoded = Dense(64, activation='relu')(input_layer)
decoded = Dense(len(selected_columns), activation='linear')(encoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')

# Cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_num = 1

for train_index, val_index in kf.split(data_scaled):
    X_train, X_val = data_scaled[train_index], data_scaled[val_index]

    # Train the Autoencoder
    autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, verbose=1)

    # Evaluate on the validation set
    val_predictions = autoencoder.predict(X_val)
    mse = mean_squared_error(X_val, val_predictions)
    print(f'Mean Squared Error on Validation Set (Fold {fold_num}): {mse}')

    # Visualize original vs. reconstructed data for a random sample from the validation set
    sample_index = np.random.randint(0, len(X_val))
    original_sample = X_val[sample_index]
    reconstructed_sample = val_predictions[sample_index]

    plt.figure(figsize=(8, 4))
    plt.plot(selected_columns, original_sample, label='Original', marker='o')
    plt.plot(selected_columns, reconstructed_sample, label='Reconstructed', marker='x')
    plt.title(f'Fold {fold_num}: Original vs. Reconstructed Data')
    plt.xlabel('Features')
    plt.ylabel('Standardized Values')
    plt.legend()
    plt.show()

    fold_num += 1
